In [6]:
import pandas as pd
import os, sys

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import warnings
warnings.filterwarnings('ignore')

from settings import *
from my_train import *
from src.model import CNN

_, datasets_augmented, size = load_data(None, METADATA_DIR_AUGMENTED_RAW, instruments=[], instruments_aug=['Guitar'])
datasets_augmented = list(map(lambda df: df[~df['augmentation'].isin(['GN_10db', 'GN_20db', 'GN_30db'])].reset_index(drop=True),datasets_augmented))

dataset_raw = datasets_augmented[0]

from imblearn.over_sampling import RandomOverSampler
def over_sample(df):
    def get_class(class_ID):
        return list(CLASSES_MAP.keys())[list(CLASSES_MAP.values()).index(class_ID)]
    oversample = RandomOverSampler(sampling_strategy='auto')
    X, y = df[['spectrogram', 'augmentation']].values, df['class_ID'].values
#     X = X.reshape(-1, 1)
    X, y = oversample.fit_resample(X, y)
    df = pd.DataFrame()
    df['spectrogram'] = pd.Series([np.array(x[0]) for x in X])
    df['augmentation'] = pd.Series([np.array(x[1]) for x in X])
    df['augmentation'] = df['augmentation'].map(lambda x: str(x))
    df['class_ID'] = pd.Series(y)
    df['class_name'] = df['class_ID'].map(lambda x: get_class(x))
    return df

def clean_shape(df):
    max_shape = df['spectrogram'].map(lambda x: x.shape).value_counts().index[0]
    print(f"The most frequent shape is {max_shape}")
    df['shape'] = df['spectrogram'].map(lambda x: x.shape)
    df = df[df['shape']==max_shape]
    df.drop(columns='shape', inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

def process_augmented(df):
    df = (df.pipe(clean_shape)
            .pipe(over_sample)
    )
    df = df[['spectrogram','class_ID', 'class_name', 'augmentation']]
    
    return df

dataset = process_augmented(dataset_raw)

np.random.seed(42)
sample = np.random.choice(dataset.index, size=int(len(dataset)*0.8), replace=False)
train_data, test_data = dataset.iloc[sample], dataset.drop(sample)

X_train = train_data['spectrogram']
y_train = train_data['class_ID']
X_test = test_data['spectrogram']
y_test = test_data['class_ID']

X_train = np.array([x.reshape( (128, 107, 1) ) for x in X_train])
X_test = np.array([x.reshape( (128, 107, 1) ) for x in X_test])

y_train = np.array(keras.utils.to_categorical(y_train, 10))
y_test_values=y_test
y_test = np.array(keras.utils.to_categorical(y_test, 10))

cnn = CNN((128, 107))
model = cnn.model

model.compile(
	optimizer="Adam",
	loss="categorical_crossentropy",
	metrics=['accuracy', precision, recall,fmeasure])

model.fit(
	x=X_train, 
	y=y_train,
    epochs=70,
    batch_size=20,
    validation_data= (X_test, y_test))

score = model.evaluate(
	x=X_test,
	y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test precision:', score[2])
print('Test recall:', score[3])
print('Test f1-score:', score[4])


The most frequent shape is (128, 107)
src.model - INFO - Initializing CNN
src.model - INFO - Input shape = (128, 107, 1)
src.model - INFO - CNN Initialized
Epoch 1/70


2023-05-22 05:45:25.667841: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8600
2023-05-22 05:45:26.566045: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


480/480 [==============================] - 11s 13ms/step - loss: 2.2164 - accuracy: 0.1643 - precision: 0.2441 - recall: 0.0186 - fmeasure: 0.0341 - val_loss: 2.0004 - val_accuracy: 0.2988 - val_precision: 0.4285 - val_recall: 0.0575 - val_fmeasure: 0.0986
Epoch 2/70
480/480 [==============================] - 6s 12ms/step - loss: 1.9687 - accuracy: 0.2874 - precision: 0.6346 - recall: 0.0967 - fmeasure: 0.1627 - val_loss: 1.6630 - val_accuracy: 0.3862 - val_precision: 0.6977 - val_recall: 0.2042 - val_fmeasure: 0.3040
Epoch 3/70
480/480 [==============================] - 6s 12ms/step - loss: 1.7113 - accuracy: 0.3948 - precision: 0.7915 - recall: 0.2221 - fmeasure: 0.3377 - val_loss: 1.5544 - val_accuracy: 0.4650 - val_precision: 0.8126 - val_recall: 0.2867 - val_fmeasure: 0.4097
Epoch 4/70
480/480 [==============================] - 6s 12ms/step - loss: 1.4857 - accuracy: 0.4866 - precision: 0.8320 - recall: 0.3099 - fmeasure: 0.4444 - val_loss: 1.2192 - val_accuracy: 0.5721 - val_prec

In [7]:
cnn.save_model("baseline_results")

src.model - INFO - Saving model
src.model - INFO - Saved model to /home/tzag/danigil/dl/guitarCR/models


In [8]:
datasets_raw, _, size = load_data(METADATA_DIR_RAW, None, instruments=['Piano', 'Accordion', 'Violin'], instruments_aug=[])

In [13]:
test_instruments = ['Piano', 'Accordion', 'Violin']
for test_data, instrument in zip(datasets_raw, test_instruments):
    X_test = test_data['spectrogram']
    reshaped = [0] * len(X_test)
    for i, x in enumerate(X_test):
        try:
            reshapen = x.reshape((128, 107, 1))
            reshaped[i] = reshapen
        except:
            pass

    idxs = [i for i in range(len(reshaped)) if reshaped[i]==0]
    
    X_test = np.array(list(filter(lambda x: x!=0, reshaped)))
    #X_test = np.array([x.reshape( (128, 107, 1) ) for x in X_test])
    y_test = test_data['class_ID']
    y_test.drop(index=idxs, inplace=True)

    y_test_values=y_test
    y_test = np.array(keras.utils.to_categorical(y_test, 10))

    score = cnn.model.evaluate(X_test,y_test)
    print(f'Test score for instrument: {instrument}')
    print('\tTest loss:', score[0])
    print('\tTest accuracy:', score[1])
    print('\tTest precision:', score[2])
    print('\tTest recall:', score[3])
    print('\tTest f1-score:', score[4])

OverflowError: cannot convert float infinity to integer